In [ ]:
#Sound Volum

In [1]:
import numpy as np
import cv2
import mediapipe as mp
import time
mp_drawing = mp.solutions.drawing_utils
hand = mp.solutions.hands
mp_hands = mp.solutions.hands.Hands(False,2,0.7,0.5)

cap=cv2.VideoCapture(0)
cap.set(3,640)
cap.set(4,480)
pTime=0

from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
volRange=volume.GetVolumeRange()

minVol=volRange[0]
maxVol=volRange[1]
volbar=400
while True:
    
    ret,image=cap.read()
    if ret==False :
        break
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = mp_hands.process(image)
    image.flags.writeable = True
    rows, cols, _ = image.shape
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks( image, hand_landmarks,hand.HAND_CONNECTIONS)
            x = [ int(landmark.x * cols) for landmark in hand_landmarks.landmark]
            y = [ int(landmark.y * rows) for landmark in hand_landmarks.landmark]
            cv2.circle(image,(x[4],y[4]),15,(255,0,0),3,-1)
            cv2.circle(image,(x[8],y[8]),15,(255,0,0),3,-1)
            cv2.line(image,(x[4],y[4]),(x[8],y[8]),(255,0,0),3)
            
            length=np.hypot(x[8]-x[4],y[8]-y[4])
            vol=np.interp(length,[20,200],[minVol,maxVol])
            volbar=np.interp(length,[20,200],[400,150])
            volume.SetMasterVolumeLevel(vol, None)
            
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_RGB2BGR)
    cv2.rectangle(image,(50,150),(85,400),(0,255,0),3)
    cv2.rectangle(image,(50,int(volbar)),(85,400),(0,255,0),cv2.FILLED)
    
    cTime=time.time()
    fps=1/(cTime-pTime)
    pTime=cTime
    cv2.putText(image,str(int(fps)),(70,50),cv2.FONT_HERSHEY_PLAIN,3,(0,0,255),3)
    cv2.imshow('MediaPipe Hands', image)
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()
cv2.destroyAllWindows()